In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend 
import os
from scipy import stats

data_path = os.path.join(os.getcwd(), 'data')
X = np.load(os.path.join(data_path, 'X_69_500.npy'), allow_pickle=True)
Y = np.load(os.path.join(data_path, 'Y_69_500.npy'), allow_pickle=True)

freq_bins = X[0].shape[0]
min_spec_len = min([x.shape[1] for x in X])
data_size = X.shape[0]

print(data_size, freq_bins, min_spec_len)
X = [tf.stack(x[:,:min_spec_len]) for x in X] # noised spectrograms
Y = [tf.stack(y[:,:min_spec_len])for i,y in enumerate(Y)] # noised - clean = noise only spectrograms

splits = train_test_split(X, Y, test_size=0.1, random_state=69)
X_train, X_test, y_train, y_test = [tf.convert_to_tensor(s) for s in splits]
for item in [X_train, X_test, y_train, y_test]:
    print(item.shape, item.dtype, type(item))

500 513 119
(450, 513, 119) <dtype: 'complex64'> <class 'tensorflow.python.framework.ops.EagerTensor'>
(50, 513, 119) <dtype: 'complex64'> <class 'tensorflow.python.framework.ops.EagerTensor'>
(450, 513, 119) <dtype: 'complex64'> <class 'tensorflow.python.framework.ops.EagerTensor'>
(50, 513, 119) <dtype: 'complex64'> <class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
from keras import layers
from tensorflow import keras
#Unet network
data_shape = (freq_bins, min_spec_len)

def unet(pretrained_weights = None,input_size = (128,128,1)):
    size_filter_in = 16
    #kernel_init = 'glorot_uniform'
    kernel_init = 'he_normal'
    activation_layer = None 
    model_input = layers.Input(shape=(frequency_bins, min_spec_len), dtype=np.complex64)
    
    conv1 = layers.Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(model_input)
    conv1 = layers.LeakyReLU()(conv1)
    conv1 = layers.Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv1)
    conv1 = layers.LeakyReLU()(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool1)
    conv2 = layers.LeakyReLU()(conv2)
    conv2 = layers.Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv2)
    conv2 = layers.LeakyReLU()(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool2)
    conv3 = layers.LeakyReLU()(conv3)
    conv3 = layers.Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv3)
    conv3 = layers.LeakyReLU()(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool3)
    conv4 = layers.LeakyReLU()(conv4)
    conv4 = layers.Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv4)
    conv4 = layers.LeakyReLU()(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = layers.Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool4)
    conv5 = layers.LeakyReLU()(conv5)
    conv5 = layers.Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv5)
    conv5 = layers.LeakyReLU()(conv5)
    drop5 = layers.Dropout(0.5)(conv5)

    up6 = layers.Conv2D(size_filter_in*8, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(drop5))
    up6 = layers.LeakyReLU()(up6)
    merge6 = tf.concatenate([drop4,up6], axis = 3)
    conv6 = layers.Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge6)
    conv6 = layers.LeakyReLU()(conv6)
    conv6 = layers.Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv6)
    conv6 = layers.LeakyReLU()(conv6)
    up7 = layers.Conv2D(size_filter_in*4, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv6))
    up7 = layers.LeakyReLU()(up7)
    merge7 = tf.concatenate([conv3,up7], axis = 3)
    conv7 = layers.Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge7)
    conv7 = layers.LeakyReLU()(conv7)
    conv7 = layers.Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv7)
    conv7 = layers.LeakyReLU()(conv7)
    up8 = layers.Conv2D(size_filter_in*2, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv7))
    up8 = layers.LeakyReLU()(up8)
    merge8 = tf.concatenate([conv2,up8], axis = 3)
    conv8 = layers.Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge8)
    conv8 = layers.LeakyReLU()(conv8)
    conv8 = layers.Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv8)
    conv8 = layers.LeakyReLU()(conv8)
    
    up9 = layers.Conv2D(size_filter_in, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv8))
    up9 = layers.LeakyReLU()(up9)
    merge9 = tf.concatenate([conv1,up9], axis = 3)
    conv9 = layers.Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge9)
    conv9 = layers.LeakyReLU()(conv9)
    conv9 = layers.Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = layers.LeakyReLU()(conv9)
    conv9 = layers.Conv2D(2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = layers.LeakyReLU()(conv9)
    conv10 = layers.Conv2D(1, 1, activation = 'tanh')(conv9)

    model = keras.Model(inputs,conv10)

    model.compile(optimizer = 'adam', loss = tf.keras.losses.Huber(), metrics = ['mae'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model
     